In [1]:
import tensorflow as tf

In [2]:
# Assume that we are receiving a batch of 40x40 pixel patches
# The associated set of labels indicates whether or not a patch contains a crack
# These labels are one-hot encoded.

patch_size  = 40
input_size  = patch_size**2
batch_size  = 32
in_channels = 1
n_factors   = 2

graph = tf.Graph()

with graph.as_default():
    with tf.name_scope('Inputs'):
        images = tf.placeholder(tf.float32, [batch_size, input_size, in_channels])
        labels = tf.placeholder(tf.float32, [batch_size, 2])
    
    with tf.name_scope('Variables'):
        mu    = tf.Variable(tf.truncated_normal(
                                shape = [input_size]))
        Sigma = tf.Variable(tf.eye(input_size))
        Phi   = tf.Variable(tf.truncated_normal(
                                shape = [input_size, n_factors]))
        
    def E_step(X, Phi, Sigma, mu):
        """
            Returns a [batch_size, n_factors] matrix.
        
            X     : [batch_size, input_size] matrix of observations.
            Phi   : [input_size, n_factors] matrix of factors.
            Sigma : [input_size, input_size] diagonal covariance matrix.
            mu    : [input_size] vector located at x's mean.
        """
        inv_Sigma = tf.diag(tf.divide(1, tf.diag_part(Sigma)))
        tmp1      = tf.tensordot(tf.tranpose(Phi), inv_Sigma, axes = [[0], [1]])
        tmp2      = tf.tensordot(tmp1, Phi, axes = [[0], [1]]) # [n_factors, n_factors]
        tmp3      = tf.tensordot(tf.inverse(tmp2 + tf.eye(n_factors)), tmp1, axes = [[0], [1]])
        Eh        = tf.tensordot(X - mu, tf.transpose(tmp3), axes = [[0], [1]])
        Ehh       = tmp3 + tf.tensordot(Eh, tf.tranpose(Eh), axes = [[0], [1]])
        return Eh, Ehh
    
    def M_step(X, mu, Phi, Eh, Ehh, batch_size):
        mu      = update_mu(mu, batch_size)
        Phi     = update_Phi(X, mu, Eh, Ehh)
        Sigma   = update_Sigma(X, mu, Phi, Eh, batch_size)
        return mu, Phi, Sigma
    
    def update_mu(mu, batch_size):
        mu_hat = tf.sum(X, axis = 0)/batch_size
        return mu_hat
    
    def update_Phi(X, mu, Eh, Ehh):
        X        = tf.expand_dims(X, axis = 0)  # X is flat and wide, depth corresponds to inidivdual samples
        Eh       = tf.expand_dims(Eh, axis = 0) # It's the same case with Eh
        tmp1     = tf.tensordot(X - mu, tf.transpose(Eh, perm = [2, 0, 1]), axis = [[]])
        Phi_hat  = tf.tensordot(tmp1, tmp2, axis = [[0], [1]])
        return Phi_hat
    
    def update_Sigma(X, mu, Phi, Eh, batch_size):
        tmp1     = tf.tensordot(X)
        
    with tf.name_scope('Optimization'):
        Eh, Ehh        = E_step(X, Phi, Sigma, mu) # [batch_size, n_factors] - Each row corresponds to a single example
        mu, Phi, Sigma = M_step(X, mu, Phi, Eh, Ehh, batch_size)
        optimize       = tf.group(mu, Phi, Sigma)
        
    with tf.name_scope('Inference'):

IndentationError: expected an indented block (<ipython-input-2-a067076e50ef>, line 44)